<a href="https://colab.research.google.com/github/njberkowitz95/geog287387/blob/master/2020SummerCL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import GEE API

In [0]:
from google.colab import auth
auth.authenticate_user()

import ee
ee.Authenticate()
ee.Initialize()

# use the Folium library for visualization
import folium

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=TDL-bCJXGossMEaBvXhtIbXIVhAiFiw5_obloUp-jyQ&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AG4SH5lDi-ma8wcoapywJvqBl-12fdm4HYTIDJabBfhVj9igDCAjGk

Successfully saved authorization token.


# Define variables 

In [0]:
# import data from earth engine asset
# missing mask
missing_18 = ee.Image('users/zhmeng0119/missing_18')
# print(missing_18)
# vectorized training sites 
training_vct = ee.FeatureCollection('users/zhmeng0119/vct_training4k_Chanthaburi')

# create the point of interest
POI = ee.Geometry.Point(101.995989,13.012611);

# use landsat 8 T1 level data
landsat8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA")

# Load up data

In [0]:
# filter the date of data by POI
landsat8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA")
l8filtered = landsat8.filterBounds(POI).filterDate('2018-01-01', '2018-12-31').first()
# print(l8filtered)

# visualize the data
# use folium to visualize the imagery.
mapid = l8filtered.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3})
map = folium.Map(location=[13.012611,101.995989])

folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='median composite',
  ).add_to(map)

map.add_child(folium.LayerControl())
map

In [0]:
# # filter the date of data by POI
# landsat8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA")
# l8filtered = landsat8.filterBounds(POI).filterDate('2018-01-01', '2018-12-31').first()
# # pansharpending
# hsv = l8filtered.select(['B4', 'B3', 'B2']).rgbToHsv();
# l8filtered = ee.Image.cat([
#   hsv.select('hue'), hsv.select('saturation'), l8filtered.select('B8')
# ]).hsvToRgb();
# # print(sharpened)

# # print(l8filtered)

# # visualize the data
# # use folium to visualize the imagery.
# mapid = l8filtered.getMapId({'bands': ['red', 'green', 'blue'], 'min': 0, 'max': 0.3})
# map = folium.Map(location=[13.012611,101.995989])

# folium.TileLayer(
#     tiles=mapid['tile_fetcher'].url_format,
#     attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
#     overlay=True,
#     name='median composite',
#   ).add_to(map)

# map.add_child(folium.LayerControl())
# map

#Preprocess data

In [0]:
# apply missing mask
masked_l8 = l8filtered.updateMask(missing_18);

#Exoprt panchromatic band 
# B8Panchromatic = masked_l8.select(['B8'])
# assexport = ee.batch.Export.image.toDrive(B8Panchromatic,
#                 description='assetExportTask', 
#                 folder='DATA_2020SummerCL',
#                 fileNamePrefix='B8Panchromatic',
#                 scale=15,
#                 maxPixels=9000000000)
# assexportid = str(assexport.id)
# print('****Exporting to Assets, task id: %s '%assexportid)
# assexport.start() 

# Export masked_l8 to drive for pansharpen process
# assexport = ee.batch.Export.image.toDrive(masked_l8.select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']),
#                 description='assetExportTask', 
#                 folder='DATA_2020SummerCL',
#                 fileNamePrefix='Landsat8Image_training_input',
#                 scale=30,
#                 maxPixels=9000000000)
# assexportid = str(assexport.id)
# print('****Exporting to Assets, task id: %s '%assexportid)
# assexport.start() 

# select the training data for each class
mangrove = training_vct.filter(ee.Filter.eq("grid_code",1))
wetland = training_vct.filter(ee.Filter.eq("grid_code",2))
pond = training_vct.filter(ee.Filter.eq("grid_code",3))
water = training_vct.filter(ee.Filter.eq("grid_code",4))
other = training_vct.filter(ee.Filter.eq("grid_code",5))
# print(other)

# Set the class "mangrove" to 1 and all others to 0
def setClass(feat):
  return feat.set("class",1)

def setOther(feat):
  return feat.set("class",0)

man = mangrove.map(setClass)
oth = wetland.map(setOther).merge(pond.map(setOther)).merge(water.map(setOther)).merge(other.map(setOther))

referenceData = man.merge(oth)

#Random Forest Classification with GEE Python Library

##Part A: regular RF for all class

In [0]:
# Sample data
trainingA = masked_l8.sampleRegions(training_vct,['grid_code'],30);
# Define a classifier and train it
classifierA = ee.Classifier.smileRandomForest(10).train(trainingA,'grid_code',['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']);
# classifierA = ee.Classifier.smileRandomForest(10).train(trainingA,'grid_code',['red', 'green', 'blue']);
# Classify masked_l8.
classifiedA = masked_l8.classify(classifierA);
# print(classifiedA)

# visualize the result
# use folium to visualize the imagery.
resultidA = classifiedA.getMapId({'bands': ['classification'], 'min': 1, 'max': 5, 'palette': ["e07a5f","3d405b","81b29a","f2cc8f","ff0e19"]})
map = folium.Map(location=[13.012611,101.995989])

folium.TileLayer(
    tiles=resultidA['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='median composite',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

##Part B: soft RF for one class (Mangrove)

In [0]:
# sample data
trainingB = masked_l8.sampleRegions(referenceData,['class'],masked_l8.get('GRID_CELL_SIZE_REFLECTIVE'));
# print(trainingB)
# define a classifier and train it
classifierB = ee.Classifier.smileRandomForest(10).setOutputMode('PROBABILITY').train(trainingB,'class',['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']);
# Classify masked_l8.
classifiedB = masked_l8.classify(classifierB);

# # visualize the result
# # use folium to visualize the imagery.
palette = ['black','blue','green','yellow','orange','red','purple']
resultidB = classifiedB.getMapId({'bands': ['classification'], 'min': 0, 'max': 1,'palette':palette})
map = folium.Map(location=[13.012611,101.995989])

folium.TileLayer(
    tiles=resultidB['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='median composite',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

In [0]:
# assexport = ee.batch.Export.image.toDrive(classifiedB,
#                 description='assetExportTask', 
#                 folder='DATA_2020SummerCL',
#                 fileNamePrefix='softRF_Mangrove',
#                 scale=30)
# assexportid = str(assexport.id)
# print('****Exporting to Assets, task id: %s '%assexportid)
# assexport.start() 

****Exporting to Assets, task id: None 


##Part C: soft RF for all classes - Batch 

#### Classes we have are mangrove(1),wetland(2),pond(3),water(4),other(5)

In [0]:
#### Classes we have are mangrove(1),wetland(2),pond(3),water(4),other(5)
Classes = {
    'mangrove':mangrove,
    'wetland':wetland,
    'pond':pond,
    'water':water,
    'other':other
}

In [0]:
def batchSoftRF(image, name):
  # create name list to index 'Classes'
  classname = ['mangrove','wetland','pond','water','other']
  del classname[classname.index(name)]

  # Set the class "name" to 1 and all others to 0
  target = Classes[name].map(setClass)
  other = Classes[classname[0]].map(setOther)
  del classname[0]
  for i in classname:
    other = other.merge(Classes[i].map(setOther))
  
  # create ref points dataset
  referenceData = target.merge(other)

  # properties of image
  spResolution = image.get('GRID_CELL_SIZE_REFLECTIVE')
  bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']

  # sample data based on ref points dataset
  training = image.sampleRegions(referenceData,['class'],30);
  # define a classifier and train it
  classifier = ee.Classifier.smileRandomForest(10).setOutputMode('PROBABILITY').train(training,'class',bands);
  # Classify masked_l8.
  classified = image.classify(classifier);

  return classified


def visualizeResult(result):
  # # visualize the result
  # # use folium to visualize the imagery.
  # palette
  palette = ['black','blue','green','yellow','orange','red','purple']
  resultid = result.getMapId({'bands': ['classification'], 'min': 0, 'max': 1,'palette':palette})
  map = folium.Map(location=[13.012611,101.995989])
  folium.TileLayer(
      tiles=resultid['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      overlay=True,
      name='median composite',
    ).add_to(map)
  map.add_child(folium.LayerControl())
  map
  return map


In [0]:
result_mangrove = batchSoftRF(masked_l8,'mangrove')
visualizeResult(result_mangrove)

# assexport = ee.batch.Export.image.toDrive(result_mangrove,
#                 description='assetExportTask', 
#                 folder='DATA_2020SummerCL',
#                 fileNamePrefix='result_mangrove_15',
#                 scale=15,
#                 maxPixels=9000000000)
# assexportid = str(assexport.name)
# print('****Exporting to Assets, task name: %s '%assexportid)
# assexport.start() 

In [0]:
result_wetland = batchSoftRF(masked_l8,'wetland')
visualizeResult(result_wetland)

# assexport = ee.batch.Export.image.toDrive(result_wetland,
#                 description='assetExportTask', 
#                 folder='DATA_2020SummerCL',
#                 fileNamePrefix='softRF_Wetland_15',
#                 scale=15,
#                 maxPixels=9000000000)
# assexportid = str(assexport.name)
# print('****Exporting to Assets, task name: %s '%assexportid)
# assexport.start() 

****Exporting to Assets, task name: None 


In [0]:
result_pond = batchSoftRF(masked_l8,'pond')
visualizeResult(result_pond)

# assexport = ee.batch.Export.image.toDrive(result_pond,
#                 description='assetExportTask', 
#                 folder='DATA_2020SummerCL',
#                 fileNamePrefix='softRF_Pond_15',
#                 scale=15,
#                 maxPixels=9000000000)
# assexportid = str(assexport.name)
# print('****Exporting to Assets, task name: %s '%assexportid)
# assexport.start() 

****Exporting to Assets, task name: None 


In [0]:
result_water = batchSoftRF(masked_l8,'water')
visualizeResult(result_water)

# assexport = ee.batch.Export.image.toDrive(result_water,
#                 description='assetExportTask', 
#                 folder='DATA_2020SummerCL',
#                 fileNamePrefix='softRF_Water_15',
#                 scale=15,
#                 maxPixels=9000000000)
# assexportid = str(assexport.name)
# print('****Exporting to Assets, task name: %s '%assexportid)
# assexport.start() 

****Exporting to Assets, task name: None 


In [0]:
result_other = batchSoftRF(masked_l8,'other')
visualizeResult(result_other)

# assexport = ee.batch.Export.image.toDrive(result_other,
#                 description='assetExportTask', 
#                 folder='DATA_2020SummerCL',
#                 fileNamePrefix='softRF_Other_15',
#                 scale=15,
#                 maxPixels=9000000000)
# assexportid = str(assexport.name)
# print('****Exporting to Assets, task name: %s '%assexportid)
# assexport.start() 

****Exporting to Assets, task name: None 


#Testing with GDAL

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# GDAL
# import osgeo
from osgeo import gdal

##Opening the File

In [0]:
dataset = gdal.Open('/content/drive/My Drive/DATA_2020SummerCL/Landsat8Image_training_input.tif')
panband = gdal.Open('/content/drive/My Drive/DATA_2020SummerCL/B8Panchromatic.tif')
if not panband:
  print("GG")
else:
  print("Data read in successed!")

Data read in successed!


### Getting Dataset Information

In [0]:
print("Driver: {}/{}".format(dataset.GetDriver().ShortName,
                            dataset.GetDriver().LongName))
print("Size is {} x {} x {}".format(dataset.RasterXSize,
                                    dataset.RasterYSize,
                                    dataset.RasterCount))
print("Projection is {}".format(dataset.GetProjection()))
geotransform = dataset.GetGeoTransform()
if geotransform:
    print("Origin = ({}, {})".format(geotransform[0], geotransform[3]))
    print("Pixel Size = ({}, {})".format(geotransform[1], geotransform[5]))

Driver: GTiff/GeoTIFF
Size is 7521 x 7693 x 7
Projection is PROJCS["WGS 84 / UTM zone 47N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",99],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32647"]]
Origin = (711105.0, 1555365.0)
Pixel Size = (30.0, -30.0)


### Fetching a Raster Band

In [0]:
band = dataset.GetRasterBand(7)
print("Band Type={}".format(gdal.GetDataTypeName(band.DataType)))

min = band.GetMinimum()
max = band.GetMaximum()
if not min or not max:
    (min,max) = band.ComputeRasterMinMax(True)
print("Min={:.3f}, Max={:.3f}".format(min,max))

if band.GetOverviewCount() > 0:
    print("Band has {} overviews".format(band.GetOverviewCount()))

if band.GetRasterColorTable():
    print("Band has a color table with {} entries".format(band.GetRasterColorTable().GetCount()))

Band Type=Float32
Min=0.007, Max=0.359


In [0]:
opt = """
<VRTDataset subClass="VRTPansharpenedDataset">
    <PansharpeningOptions>
        <PanchroBand>
            <SourceFilename relativeToVRT="1">panchromatic.tif</SourceFilename>
            <SourceBand>1</SourceBand>
        </PanchroBand>
        <SpectralBand dstBand="1">
            <SourceFilename relativeToVRT="1">multispectral.tif</SourceFilename>
            <SourceBand>1</SourceBand>
        </SpectralBand>
        <SpectralBand dstBand="2">
            <SourceFilename relativeToVRT="1">multispectral.tif</SourceFilename>
            <SourceBand>2</SourceBand>
        </SpectralBand>
        <SpectralBand dstBand="3">
            <SourceFilename relativeToVRT="1">multispectral.tif</SourceFilename>
            <SourceBand>3</SourceBand>
        </SpectralBand>
    </PansharpeningOptions>
</VRTDataset>
"""